# # To-Do:
1. Construct baseline model + saving specification 
2. Calculate epoch specifications 
3. Figure out how to tie graph to GPU device 
4. start baseline training 

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from trackml.dataset import load_event, load_dataset
from trackml.randomize import shuffle_hits
from trackml.score import score_event
import os
import math 
%run utils.ipynb

In [2]:
N_STEPS = 19
INPUTS_DIM = 5
OUTPUTS_DIM = 3
LSTM_NEURONS = 400
LEARN_RATE = 0.0005 
BATCH_SIZE = 50
n_iterations = 100000000

In [3]:

global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name = 'global_step')
with tf.device('/device:GPU:1'):
    
    X = tf.placeholder(tf.float32, [None, N_STEPS, INPUTS_DIM], name = 'input_ph' )
    Y = tf.placeholder(tf.float32, [None, N_STEPS, OUTPUTS_DIM], name = 'label_ph' )
    #constructing model

    lstm = tf.contrib.rnn.LSTMCell(num_units = LSTM_NEURONS, use_peepholes = True)
    #adding FC layer
    lstm_cell = tf.contrib.rnn.OutputProjectionWrapper(lstm, output_size = OUTPUTS_DIM)
    rnn_outputs, hidden_states = tf.nn.dynamic_rnn(lstm_cell, X, dtype = tf.float32)

    #prediction = tf.Variable(rnn_outputs([BATCH_SIZE, N_STEPS, OUTPUTS_DIM]), name = 'prediction')
    #print_op might not be necessary

    loss_diff = tf.reduce_sum(tf.square(tf.subtract(rnn_outputs, Y)), 2)
    mask = tf.sign(tf.reduce_max(tf.abs(Y), 2))
    mask_loss = mask*loss_diff
    batch_loss = tf.reduce_mean(mask_loss)

    train = tf.train.AdamOptimizer(learning_rate=LEARN_RATE)
    train_op = train.minimize(batch_loss, global_step=global_step)

print_op = tf.Print(rnn_outputs, [rnn_outputs], name = 'print_op')
init=tf.global_variables_initializer()
    #data_itr = next_batch(N_STEPS, BATCH_SIZE, INPUTS_DIM)


data_itr = next_batch(N_STEPS, BATCH_SIZE, INPUTS_DIM, simple = True, full_tracks = True, ideal = False)




with tf.name_scope('summaries'):
    tf.summary.scalar('loss', batch_loss)
    tf.summary.histogram('histogram_loss', batch_loss)
    summary_op = tf.summary.merge_all()

with tf.Session(config=tf.ConfigProto(log_device_placement=True)) as baseline_hit_cylinder:
    init.run()
    initial_step = global_step.eval()
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter('./logs/baseline-cylinder-hits/', baseline_hit_cylinder.graph)
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('./checkpoints/baseline-cylinder-hits/'))
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(baseline_hit_cylinder, ckpt.model_checkpoint_path)
        
    for iteration in range(n_iterations):
        x_batch, y_batch, _, __ = next(data_itr)
        
        ___, loss_batch, summary, test_print = baseline_test_sess.run([train_op, batch_loss, summary_op, print_op], feed_dict = {X:x_batch, Y:y_batch})
        writer.add_summary(summary, global_step = iteration)
        #print('input:')
        #print(x_batch)
        #print('pred:')
        #print(test_print)
        if iteration %100 ==0:
            mse = batch_loss.eval(feed_dict={X:x_batch, Y:y_batch})
            print(iteration, "\tMSE:", mse)
            saver.save(baseline_hit_cylinder, './checkpoints/baseline-cylinder-hits/', iteration)

Instructions for updating:
Use the retry module or similar alternatives.


InternalError: Failed to create session.